# Text Summarization. Homework

Всем привет! Это домашка по суммаризации текста.

На семинаре мы рассмотрели базовые модели для суммаризации текста. Попробуйте теперь улучшить два метода: TextRank и Extractive RNN. Задание достаточно большое и требует хорошую фантазию, тут можно эксперементировать во всю.

Для сдачи заданий надо получить определенное качество по test-у:

- 1 задание: 0.27 BLEU
- 2 задание: 0.3 BLEU

Если ваш подход пробивает это качество – задание считается пройденным. Плюсом будет описание того, почему вы решили использовать то или иное решение. 

Датасет: gazeta.ru

**P.S.** Возможно, в датасете находятся пустые данные. Проверьте эту гипотезу, и если надо, сделайте предобратоку датасета.


`Ноутбук создан на основе семинара Гусева Ильи на кафедре компьютерной лингвистики МФТИ.`

Загрузим датасет и необходимые библиотеки

In [4]:
# !wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
# !wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
# !wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [2]:
# !pip3 install --upgrade razdel allennlp torch fasttext OpenNMT-py networkx pymorphy2 nltk rouge==0.3.1 summa
# !pip3 install transformers youtokentome catalyst

    100% |████████████████████████████████| 7.6MB 91kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/13/70/54e9fb010fe1547bc4774716f11ececb81ae5b306c05f090f4461ee13205/torch-1.5.0-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 71kB 88kB/s eta 0:00:01
    100% |████████████████████████████████| 194kB 134kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/41/8f/dd6a8e85946def36e4f2c69c84219af0fa5e832b018c970e92f2ad337e45/networkx-2.4-py3-none-any.whl
    100% |████████████████████████████████| 51kB 126kB/s ta 0:00:01
    100% |████████████████████████████████| 1.4MB 158kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 242kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/f2/28/2a03252dfb9ebf377f40fba6a7841b47083260bf8bd8e737b0c6952df83f/Flask-1.1.2-py2.py3-none-any.whl
    100% |████████████████████████████████| 40kB 165kB/s ta 0:00:01
    100% |████████████████████████████████| 30.

    100% |████████████████████████████████| 61kB 120kB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 116kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/03/27/e35e7c6e6a52fab9fcc64fc2b20c6b516eba930bb02b10ace3b38200d3ab/numpy-1.18.4-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 6.9MB 97kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/89/e3/d576f6f02bc75bacbc3d42494e8f1d063c95617d86648dba243c2cb3963e/pybind11-2.5.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 583kB 110kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/65/eb/1f97cb97bfc2390a276969c6fae16075da282f5058082d4cb10c6c5c1dba/six-1.14.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 51kB 159kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/1d/fd/4f3ca1516cbb3713259ef229abd9314bba0077ef6070285dde0dd1ed21b2/tensorboard-2.2.1-py3-none-any.whl
    100% |██████████

  Using cached https://files.pythonhosted.org/packages/e1/e5/df302e8017440f111c11cc41a6b432838672f5a70aa29227bf58149dc72f/urllib3-1.25.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/e3/afebe61c546d18fb1709a61bee788254b40e736cff7271c7de5de2dc4128/idna-2.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/57/2b/26e37a4b034800c960a00c4e1b3d9ca5d7014e983e6e729e33ea2f36426c/certifi-2020.4.5.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
    100% |████████████████████████████████| 51kB 104kB/s a 0:00:011
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/23/147de658aabbf968324551ea22c0c13a00284c4ef49a77002e91f79657b7/kiwisolver-1.2.0-cp36-cp

    100% |████████████████████████████████| 92kB 95kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/2d/68/106af3ae51daf807e9cdcba6a90e518954eb8b70341cee52995540a53ead/Pygments-2.6.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/15/a1/522dccd23e5d2e47aed4b6a16795b8213e3272c7506e625f2425ad025a19/Babel-2.8.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 92kB 136kB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 199kB/s a 0:00:01
  Using cached https://files.pythonhosted.org/packages/81/44/8a15e45ffa96e6cf82956dd8d7af9e666357e16b0d93b253903475ee947f/docutils-0.16-py2.py3-none-any.whl
    100% |████████████████████████████████| 102kB 104kB/s  0:00:011
  Using cached https://files.pythonhosted.org/packages/95/de/214830a981892a3e286c3794f41ae67a4495df1108c3da8a9f62159b9a9d/pyasn1_modules-0.2.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/e5/38518af393f7c214357079ce67a3173079368

    100% |████████████████████████████████| 645kB 137kB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 102kB/s a 0:00:01
  Using cached https://files.pythonhosted.org/packages/64/05/ac1e89cd9533c66e10e286d23dc2c4d042abee177bba9d1be738f55fa10d/catalyst-20.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/83/71a2ee6158bb9f39a90c0dea1637f81d5eef866e188e1971a1b1ab01a35a/filelock-3.0.12-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3b/88/49e772d686088e1278766ad68a463513642a2a877487decbd691dec02955/sentencepiece-0.1.90-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c9/40/058b12e8ba10e35f89c9b1fdfc2d4c7f8c05947df2d5eb3c7b258019fda0/tqdm-4.46.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/14/e5/a26eb4716523808bb0a799fcfdceb6ebf77a18169d9591b2f46a9adb87d9/tokenizers-0.7.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/pac

  Using cached https://files.pythonhosted.org/packages/7b/b8/88def36e74bee9fce511c9519571f4e485e890093ab7442284f4ffaef60b/google_auth_oauthlib-0.4.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/a3/2695fdcda305ea85c43ebd2a4d1429f3c7e897c7cf9045a8c378e1115a15/google_auth-1.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8c/23/848298cccf8e40f5bbb59009b32848a4c38f4e7f3364297ab3c3e2e2cd14/wheel-0.34.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/cd/a0c1f9e4582ea64dddf76c1b808b318d01e3b858a51c715bffab1016ecc7/tensorboard_plugin_wit-1.6.0.post3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cc/94/5f7079a0e00bd6863ef8f1da638721e9da21e5bacee597595b318f71d62e/Werkzeug-1.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/35/6c/d1cf8521f94b332851cf95dbf679a72271ba85b367e2283b9f5f6cdf98bc/protobuf-3.12.0-cp36-cp36m-manylinux1_x86_64.whl
  Using c

In [1]:
import random
import numpy as np
import pandas as pd

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(eval(line)) # Simple hack
    records = pd.DataFrame(records)
    if sort_by_date:
        records = records.sort("date")
    if shuffle:
        records = records.sample(frac=1)
    return records

In [2]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

## 1 задание: TextRank (порог: 0.27 BLEU)

TextRank - unsupervised метод для составления кратких выжимок из текста. 
Описание метода:

1. Сплитим текст по предложениям
2. Считаем "похожесть" предложений между собой
3. Строим граф предложений с взвешенными ребрами
4. С помощью алгоритм PageRank получаем наиболее важные предложения, на основе которых делаем summary.

Функция похожести можно сделать и из нейросетевых(или около) моделек: FastText, ELMO и BERT. Выберете один метод, загрузите предобученную модель и с ее помощью для каждого предложениия сделайте sentence embedding. С помощью косинусной меры определяйте похожесть предложений.

Предобученные модели можно взять по [ссылке](http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html).

In [3]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [4]:
import razdel

def calc_lead_n_score(records, n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for text, summary in records[['text', 'summary']].values[:nrows]:
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: из-за того, что кредиторы стали оценивать заемщиков по долговой нагрузке, увеличилось число заявок на займы в микрофинансовых организациях. спрос за десять месяцев вырос на 20% по сравнению с прошлым годом. при этом одобряются отнюдь не все заявки, и часть россиян пойдут к нелегальным кредиторам, чтобы перехватить денег до зарплаты, предупреждают эксперты.
Hyp: спрос россиян на займы в мфо растет. по данным на 1 ноября этого года, наши сограждане оформили 170 млн заявок на получение займов в микрофинансовых организациях. об этом рассказал «газете.ru» генеральный директор финансового маркета юником24 юрий кудряков. это выше на 20%, чем в соответствующий период прошлого года. так, годом ранее россияне отправили около 141 млн заявок за 10 месяцев, в 2017 году — 107-110 млн. в целом эксперт считает, что по итогам этого года физические лица подадут порядка 200 млн заявок на получение займов. при этом реально будет выдано всего 32-33 млн займов. увеличившийся спрос на займы 

In [5]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs

bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = '/storage/Study/neuro2/hw5/rubert_cased_L-12_H-768_A-12_pt'

m = build_model(bert_config)

[nltk_data] Downloading package punkt to /home/care1e55/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/care1e55/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [6]:
def document_to_vec(words, embeddings, dim=768):
    n_known = 0
    result = np.array([0] * dim, dtype=float)
    
    for word in words:
        result += embeddings([word])[1][0][0]
        n_known += 1

    if n_known != 0:
        return result/n_known
    else:
        return result

In [9]:
document_to_vec(['And mine too, please!'], m, dim=768)

array([-1.63580272e-02, -2.99739808e-01,  4.27470952e-01,  2.82085747e-01,
        2.20809788e-01,  6.00778222e-01, -2.38174558e-01, -4.78468537e-01,
       -8.75081941e-02,  8.52746725e-01,  1.19910038e+00, -2.81395972e-01,
       -3.30129862e-01,  3.23548526e-01,  3.03210706e-01,  8.76909316e-01,
       -1.57103479e-01, -3.42637263e-02, -4.50852215e-01,  1.12513840e+00,
       -4.93100941e-01,  1.19064689e-01,  1.50534928e-01, -7.53967583e-01,
        1.47298753e-01,  8.76324594e-01,  6.48700237e-01,  2.52484083e-01,
        9.94792655e-02, -5.42144299e-01,  3.85844767e-01,  4.96216118e-01,
       -5.40555298e-01,  8.89666915e-01,  4.15382951e-01, -1.82250619e-01,
        4.37541902e-01,  7.66776204e-01, -1.10241091e+00, -9.74568546e-01,
        8.56557041e-02, -3.52700233e-01, -9.17057320e-02,  3.38391542e-01,
       -6.72827065e-02, -5.42805158e-02,  5.43608665e-01, -1.43660262e-01,
       -1.20182073e+00,  1.39885455e-01,  1.25244766e-01,  1.83576524e-01,
        1.03011835e+00, -

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# cosine simularity и ембедить предложения
def embeding_cos_similarity(words1, words2):
    words1 = document_to_vec(words1, m, dim=768)
    words2 = document_to_vec(words2, m, dim=768)
    if not len(words1) or not len(words2):
        return 0.0
    return cosine_similarity([words1], [words2])[0,0]

In [11]:
from itertools import combinations
import networkx as nx
import numpy as np
import pymorphy2


def unique_words_similarity(words1, words2):
    '''
    Функция подсчёта близости предложений на основе пересечения слов
    ''' 
    words1 = set(words1)
    words2 = set(words2)
    if not len(words1) or not len(words2):
        return 0.0
    return len(words1.intersection(words2))/(np.log10(len(words1)) + np.log10(len(words2)))

def gen_text_rank_summary(text, calc_similarity=embeding_cos_similarity, summary_part=0.1, lower=True, morph=None):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    n_sentences = len(sentences)

    # Токенизируем предложения
    sentences_words = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]

    # При необходимости лемматизируем слова
    if morph is not None:
        sentences_words = [[morph.parse(word)[0].normal_form for word in words] for words in sentences_words]  
    
    
    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2)
    scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    pr = nx.pagerank(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])

    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    return predicted_summary

def calc_text_rank_score(records, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, nrows=5770, morph=None):
    references = []
    predictions = []

    for text, summary in records[['text', 'summary']].values[:nrows]:
        summary = summary if not lower else summary.lower()
        references.append(summary)

        predicted_summary = gen_text_rank_summary(text, calc_similarity, summary_part, lower, morph=morph)
        text = text if not lower else text.lower()
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

morph = pymorphy2.MorphAnalyzer()
calc_text_rank_score(test_records)
calc_text_rank_score(test_records, morph=morph)

Count: 5770
Ref: киев должен прямо ответить, намерен ли он выполнять dторое минское соглашение — с таким заявлением обратился президент россии владимир путин к своему украинскому коллеге владимиру зеленскому в ходе телефонного разговора. риторика российского лидера обусловлена рядом недавних высказываний украинских властей относительно своего видения урегулирования конфликта в донбассе.
Hyp: в ходе беседы президент россии особо отметил важность полной и безусловной реализации также и положений, принятых лидерами россии, украины, германии и франции на саммите в «нормандском формате», прошедшем в париже 9 декабря прошлого года. «эти вопросы должны решаться в режиме прямого политического диалога между киевом и донбассом  киевская власть не сможет восстановить отношения с донбассом, пока не начнет разговаривать с жителями донбасса», — подчеркнул грызлов.
BLEU:  0.27748426882615645
ROUGE:  {'rouge-1': {'f': 0.16185144539360458, 'p': 0.1349236513608401, 'r': 0.21801411088292716}, 'rouge-2': 

## 2 Задание: Extractive RNN (порог: 0.3 BLEU)

Второй метод, который вам предлагается улучшить – поиск предложений для summary с помощью RNN. В рассмотренной методе мы использовали LSTM для генерации sentence embedding. Попробуйте использовать другие архитектуры: CNN, Transformer; или добавьте предобученные модели, как и в первом задании.

P.S. Тут предполагается, что придется изменять много кода в ячееках (например, поменять токенизацию). 

### Модель

Картинка для привлечения внимания:

![img](https://storage.googleapis.com/groundai-web-prod/media%2Fusers%2Fuser_14%2Fproject_398421%2Fimages%2Farchitecture.png)

Статья с оригинальным методом:
https://arxiv.org/pdf/1611.04230.pdf

Список вдохновения: 
- https://towardsdatascience.com/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-d2ee64b9dd0b Пример того, как можно применять CNN в текстовых задачах
- https://arxiv.org/pdf/1808.08745.pdf Очень крутой метод генерации summary без Transformers
- https://towardsdatascience.com/super-easy-way-to-get-sentence-embedding-using-fasttext-in-python-a70f34ac5b7c – простой метод генерации sentence embedding
- https://towardsdatascience.com/fse-2b1ffa791cf9 – Необычный метод генерации sentence embedding
- https://github.com/UKPLab/sentence-transformers – BERT предобученный для sentence embedding

P.S. Выше написанные ссылки нужны только для разогрева вашей фантазии, можно воспользоваться ими, а можно придумать свой.

Комментарий к заданию:
Если посмотреть на архитектуру ~~почти~~ SummaRuNNer, то в ней есть два главных элемента: первая часть, которая читает предложения и возвращает векторы на каждое предложение, и вторая, которая выбирает предложения для суммаризации. Вторую часть мы не трогаем, а первую меняем. На что меняем – как вы решите. Главное: она должна иметь хорошее качество и встроиться в текущую модель.

In [5]:
import copy
import random

def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    score = -1.0
    summaries = []
    for _ in range(min(n_sentences, 2)):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [6]:
from tqdm import tqdm_notebook as tqdm

def calc_oracle_score(records, nrows=1000, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
  
    for text, summary in tqdm(records[['text', 'summary']].values[:nrows]):
        summary = summary if not lower else summary.lower()
        references.append(summary)
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge))
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_oracle_score(train_records)

/home/care1e55/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Count: 1000
Ref: президенты россии и турции владимир путин и реджеп тайип эрдоган обсудили вопрос признания сша иерусалима столицей израиля по телефону. российский лидер поддержал позицию главы турции и подчеркнул, что решение сша способствует росту напряженности в регионе.
Hyp: стороны также отметили недопустимость дальнейшей эскалации напряженности в регионе и подтвердили приверженность россии и турции достижению справедливого и жизнеспособного решения ближневосточного кризиса на основе соответствующих резолюций совета безопасности и генеральной ассамблеи оон. пресс-секретарь президента россии дмитрий песков заявил, что решение президента сша дональда трампа о признании иерусалима столицей израиля не способствует ближневосточному урегулированию.
BLEU:  0.415488069082347
ROUGE:  {'rouge-1': {'f': 0.31210835269088893, 'p': 0.37392704527998943, 'r': 0.28535927895414975}, 'rouge-2': {'f': 0.1666899310777806, 'p': 0.20579618805246494, 'r': 0.15013773493648735}, 'rouge-l': {'f': 0.2660650

In [7]:
# import pickle

# with open('train_records.pickle', 'wb') as f:
#     pickle.dump(train_records, f)

# # with open('train_records.pickle', 'rb') as f:
# # data_new = pickle.load(f)

## (!)
Если надо, поменяйте код загрузки токенизатора

In [8]:
import os

import youtokentome as yttm

def train_bpe(records, model_path, model_type="bpe", vocab_size=10000, lower=True):
    temp_file_name = "temp.txt"
    with open(temp_file_name, "w") as temp:
        for text, summary in records[['text', 'summary']].values:
            if lower:
                summary = summary.lower()
                text = text.lower()
            if not text or not summary:
                continue
            temp.write(text + "\n")
            temp.write(summary + "\n")
    yttm.BPE.train(data=temp_file_name, vocab_size=vocab_size, model=model_path)

train_bpe(train_records, "BPE_model.bin")

In [9]:
import youtokentome as yttm

bpe_processor = yttm.BPE('BPE_model.bin')
bpe_processor.encode(["октябрь богат на изменения"], output_type=yttm.OutputType.SUBWORD)

[['▁окт', 'ябрь', '▁бога', 'т', '▁на', '▁изменения']]

## (!)
Если надо, поменяйте код словаря

In [10]:
from collections import Counter
from typing import List, Tuple
import os

class Vocabulary:
    def __init__(self, bpe_processor):
        self.index2word = bpe_processor.vocab()
        self.word2index = {w: i for i, w in enumerate(self.index2word)}
        self.word2count = Counter()

    def get_pad(self):
        return self.word2index["<PAD>"]

    def get_sos(self):
        return self.word2index["<SOS>"]

    def get_eos(self):
        return self.word2index["<EOS>"]

    def get_unk(self):
        return self.word2index["<UNK>"]
    
    def has_word(self, word) -> bool:
        return word in self.word2index

    def get_index(self, word):
        if word in self.word2index:
            return self.word2index[word]
        return self.get_unk()

    def get_word(self, index):
        return self.index2word[index]

    def size(self):
        return len(self.index2word)

    def is_empty(self):
        empty_size = 4
        return self.size() <= empty_size

    def reset(self):
        self.word2count = Counter()
        self.index2word = ["<pad>", "<sos>", "<eos>", "<unk>"]
        self.word2index = {word: index for index, word in enumerate(self.index2word)}

In [11]:
vocabulary = Vocabulary(bpe_processor)
vocabulary.size()

10000

In [12]:
from rouge import Rouge
import razdel

def add_oracle_summary_to_records(records, max_sentences=30, lower=True, nrows=1000):
    rouge = Rouge()
    sentences_ = []
    oracle_sentences_ = []
    oracle_summary_ = []
    records = records.iloc[:nrows].copy()

    for text, summary in tqdm(records[['text', 'summary']].values):
        summary = summary.lower() if lower else summary
        sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
        oracle_summary, sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge),
                                                                         lower=lower, max_sentences=max_sentences)
        sentences_ += [sentences]
        oracle_sentences_ += [list(sentences_indicies)]
        oracle_summary_ += [oracle_summary]
    records['sentences'] = sentences_
    records['oracle_sentences'] = oracle_sentences_
    records['oracle_summary'] = oracle_summary_
    return records

ext_train_records = add_oracle_summary_to_records(train_records, nrows=4096)
ext_val_records = add_oracle_summary_to_records(val_records, nrows=256)
ext_test_records = add_oracle_summary_to_records(test_records, nrows=256)

/home/care1e55/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [14]:
# import pickle

# # with open('ext_train_records.pickle', 'wb') as f:
# #     pickle.dump(ext_train_records, f)

# # with open('ext_val_records.pickle', 'wb') as f:
# #     pickle.dump(ext_val_records, f)

# # with open('ext_test_records.pickle', 'wb') as f:
# #     pickle.dump(ext_test_records, f)

    
# with open('ext_train_records.pickle', 'rb') as f:
#     ext_train_records = pickle.load(f)

# with open('ext_val_records.pickle', 'rb') as f:
#     ext_val_records = pickle.load(f)

# with open('ext_test_records.pickle', 'rb') as f:
#     ext_test_records = pickle.load(f)


In [13]:
ext_train_records.iloc[0]

url                 https://www.gazeta.ru/politics/2011/07/06_a_36...
text                Вопрос об округе, от которого Валентина Матвие...
title                                   Матвиенко сдался первый совет
summary             Валентина Матвиенко может стать муниципальным ...
date                                              2011-07-06 15:00:04
sentences           [вопрос об округе, от которого валентина матви...
oracle_sentences                                               [0, 9]
oracle_summary      вопрос об округе, от которого валентина матвие...
Name: 41368, dtype: object

In [14]:
import re
import nltk
import string
from nltk.corpus import stopwords
stopWords = set(stopwords.words('russian'))

def text_prepare(text):
    """
        text: a string
        
    """
    text = text.lower()
    text = re.sub(r'[{}]'.format(string.punctuation), '', text)
    text = re.sub('[^А-Яа-я\. ]', '', text)
    stopWords = set(stopwords.words('english'))
    for stopWord in stopWords:
        # text = text.replace(stopWord, '')
        text = re.sub('^{}$'.format(stopWord), '', text)
    return text

In [15]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from torch.utils import data


class ExtDataset(data.Dataset):
    def __init__(self, records, vocabulary, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = records.shape[0]
        self.bpe_processor = bpe_processor
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.records.shape[0]

    def __getitem__(self, idx):
        cur_record = self.records.iloc[idx]
        inputs = list(map(lambda x: text_prepare(x), cur_record['sentences']))
        inputs = list(map(lambda x: [i.text for i in razdel.tokenize(x[:self.max_sentence_length])], inputs))
        inputs = list(map(lambda x: [morph.parse(word)[0].normal_form for word in x] , inputs))
        outputs = [int(i in cur_record['oracle_sentences']) for i in range(len(cur_record['sentences']))]
        return {'inputs': inputs, 'outputs': outputs}


In [16]:
train_dataset = ExtDataset(ext_train_records, vocabulary)
test_dataset = ExtDataset(ext_test_records, vocabulary)

In [17]:
# import pickle

# with open('ext_train_records.pickle', 'wb') as f:
#     pickle.dump(ext_train_records, f)

# with open('ext_val_records.pickle', 'wb') as f:
#     pickle.dump(ext_val_records, f)

# with open('ext_test_records.pickle', 'wb') as f:
#     pickle.dump(ext_test_records, f)

    
# # with open('train_records.pickle', 'rb') as f:
# # data_new = pickle.load(f)

In [18]:
train_dataset[0]

{'inputs': [['вопрос',
   'о',
   'округа',
   'от',
   'который',
   'валентин',
   'матвиенко',
   'плата'],
  ['полномочие', 'утратить', 'муниципальный', 'совет', 'в', 'посёлок', 'а'],
  ['в', 'заявление', 'баранников', 'написать', 'мой', 'решение', 'добро'],
  ['оно', 'вызвать', 'желание', 'предоставить', 'возможность', 'и', 'с'],
  ['ирина', 'баранников', 'в', 'девичество', 'волох', 'год', 'она', 'чле'],
  ['в', 'муниципалитет', 'она', 'возглавлять', 'комиссия', 'по', 'борьба'],
  ['баллотироваться', 'в', 'депутат', 'от', 'справедливый', 'россия'],
  ['она', 'отец', 'тоже', 'единоросс', 'главный', 'федеральный', 'инспечь'],
  ['представлять', 'он', 'в', 'год', 'член', 'правительство', 'санкт'],
  ['теперь', 'в', 'округа', 'быть', 'объявить', 'досрочный', 'выбор', 'год'],
  ['у', 'мы', 'быть', 'депутат', 'вместо', 'положить', 'считать'],
  ['после', 'тот', 'как', 'ирина', 'баранников', 'написать', 'заявить'],
  ['по', 'закон', 'совет', 'потерять', 'легитимность'],
  ['в', 'избирате

In [19]:
fse_train_dataset = []
fse_test_dataset = []
fse_train_dataset = [j for i in tqdm(train_dataset) for j in i['inputs']]
fse_test_dataset = [j for i in tqdm(test_dataset) for j in i['inputs']]

/home/care1e55/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/home/care1e55/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [20]:
# import pickle

# # with open('fse_train_dataset.pickle', 'wb') as f:
# #     pickle.dump(fse_train_dataset, f)

# # with open('fse_test_dataset.pickle', 'wb') as f:
# #     pickle.dump(fse_test_dataset, f)
    
# with open('fse_train_dataset.pickle', 'rb') as f:
#     fse_train_dataset = pickle.load(f)

# with open('fse_test_dataset.pickle', 'rb') as f:
#     fse_test_dataset = pickle.load(f)

In [21]:
len(fse_train_dataset)

116810

In [22]:
from fse import IndexedList
indexed_tokenized_sentences_train = IndexedList(fse_train_dataset)
indexed_tokenized_sentences_test = IndexedList(fse_test_dataset)
indexed_tokenized_sentences_train[2]

(['в', 'заявление', 'баранников', 'написать', 'мой', 'решение', 'добро'], 2)

In [23]:
import gensim
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(
    fname='ft_native_300_ru_wiki_lenta_lower_case.vec', binary=False)

In [37]:
from fse.models import Average
fse_model = Average(wv_embeddings, workers=2)
fse_model.train(indexed_tokenized_sentences_train)
fse_model.train(indexed_tokenized_sentences_test)

(7411, 53082)

In [38]:
# train_dataset[0]['inputs']

In [39]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


from torch.utils import data


class ExtDataset(data.Dataset):
    def __init__(self, records, vocabulary, fse_model=fse_model, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = records.shape[0]
        self.bpe_processor = bpe_processor
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        self.model = fse_model
        
    def __len__(self):
        return self.records.shape[0]

    def __getitem__(self, idx):
        cur_record = self.records.iloc[idx]
        inputs = list(map(lambda x: text_prepare(x), cur_record['sentences']))
        inputs = list(map(lambda x: [i.text for i in razdel.tokenize(x[:self.max_sentence_length])], inputs))
        inputs = list(map(lambda x: [morph.parse(word)[0].normal_form for word in x] , inputs))
        inputs = self.model.infer(IndexedList(inputs))
        outputs = [int(i in cur_record['oracle_sentences']) for i in range(len(cur_record['sentences']))]
        return {'inputs': inputs, 'outputs': outputs}


In [40]:
train_dataset = ExtDataset(ext_train_records, vocabulary, fse_model)
test_dataset = ExtDataset(ext_test_records, vocabulary, fse_model)

In [41]:
len(train_dataset[0]['inputs'][0])

300

## (!)
Если надо, поменяйте код генератора датасета и батчевалки

In [42]:
# Это батчевалка
def collate_fn(records):
    max_length = max(len(sentence) for record in records for sentence in record['inputs'])
    max_sentences = max(len(record['outputs']) for record in records)

    new_inputs = torch.zeros((len(records), max_sentences, max_length))
    new_outputs = torch.zeros((len(records), max_sentences))
    for i, record in enumerate(records):
        for j, sentence in enumerate(record['inputs']):
            new_inputs[i, j, :len(sentence)] += np.array(sentence)
        new_outputs[i, :len(record['outputs'])] += np.array(record['outputs'])
    return {'features': new_inputs.type(torch.FloatTensor), 'targets': new_outputs}

In [43]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack


class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 token_embedding_dim=300,
                 sentence_encoder_hidden_size=300,
                 hidden_size=128,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        super(SentenceTaggerRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.classify = nn.Sigmoid()
        self.tanh_layer = nn.Tanh()

    def forward(self, inputs, hidden=None):
        batch_size = inputs.size(0)
        sentences_count = inputs.size(1)
        tokens_count = inputs.size(2)
        inputs = inputs.reshape(-1, tokens_count)
#         embedded_sentences = self.sentence_encoder(inputs)
        embedded_sentences = inputs
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs)
        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        content = self.content_linear_layer(outputs).squeeze(2)
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2)
        out = self.classify(content + salience)

#         print(F.softmax(out))
        return F.softmax(out)

model = SentenceTaggerRNN(vocabulary.size())

In [44]:
torch.cuda.empty_cache()
torch.cuda.is_available()

True

### Обучение

In [45]:
import catalyst
from catalyst.dl.runner import SupervisedRunner

device = torch.device('cuda')

loaders = {
    'train': data.DataLoader(ExtDataset(ext_train_records, vocabulary, fse_model), batch_size=128, collate_fn=collate_fn),
    'valid': data.DataLoader(ExtDataset(ext_val_records, vocabulary, fse_model), batch_size=128, collate_fn=collate_fn),
    'test': data.DataLoader(ExtDataset(ext_test_records, vocabulary, fse_model), batch_size=128, collate_fn=collate_fn),
}

lr = 1e-3
num_epochs = 10

optimizer  = torch.optim.Adam(model.parameters(), lr=lr)
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCELoss()
runner = SupervisedRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    loaders=loaders,
    logdir='./logs',
    num_epochs=num_epochs,
    criterion=criterion,
    verbose=True
)




1/10 * Epoch (train):   0% 0/32 [00:00<?, ?it/s]

1/10 * Epoch (train):   0% 0/32 [00:05<?, ?it/s, loss=0.228]

1/10 * Epoch (train):   3% 1/32 [00:05<03:03,  5.93s/it, loss=0.228]

/home/care1e55/.local/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.





1/10 * Epoch (train):   3% 1/32 [00:11<03:03,  5.93s/it, loss=0.231]

1/10 * Epoch (train):   6% 2/32 [00:11<02:58,  5.96s/it, loss=0.231]

1/10 * Epoch (train):   6% 2/32 [00:17<02:58,  5.96s/it, loss=0.227]

1/10 * Epoch (train):   9% 3/32 [00:17<02:52,  5.94s/it, loss=0.227]

1/10 * Epoch (train):   9% 3/32 [00:23<02:52,  5.94s/it, loss=0.226]

1/10 * Epoch (train):  12% 4/32 [00:23<02:45,  5.90s/it, loss=0.226]

1/10 * Epoch (train):  12% 4/32 [00:29<02:45,  5.90s/it, loss=0.227]

1/10 * Epoch (train):  16% 5/32 [00:29<02:39,  5.89s/it, loss=0.227]

1/10 * Epoch (train):  16% 5/32 [00:35<02:39,  5.89s/it, loss=0.225]

1/10 * Epoch (train):  19% 6/32 [00:35<02:31,  5.84s/it, loss=0.225]

1/10 * Epoch (train):  19% 6/32 [00:41<02:31,  5.84s/it, loss=0.223]

1/10 * Epoch (train):  22% 7/32 [00:41<02:25,  5.82s/it, loss=0.223]

1/10 * Epoch (train):  22% 7/32 [00:46<02:25,  5.82s/it, loss=0.216]

1/10 * Epoch (train):  25% 8/32 [00:46<02:19,  5.80s/it, loss=0.216]

1/10 * Epoch (trai

2/10 * Epoch (train):  41% 13/32 [01:15<01:49,  5.78s/it, loss=0.206]

2/10 * Epoch (train):  41% 13/32 [01:21<01:49,  5.78s/it, loss=0.210]

2/10 * Epoch (train):  44% 14/32 [01:21<01:44,  5.78s/it, loss=0.210]

2/10 * Epoch (train):  44% 14/32 [01:26<01:44,  5.78s/it, loss=0.210]

2/10 * Epoch (train):  47% 15/32 [01:26<01:37,  5.75s/it, loss=0.210]

2/10 * Epoch (train):  47% 15/32 [01:32<01:37,  5.75s/it, loss=0.209]

2/10 * Epoch (train):  50% 16/32 [01:32<01:31,  5.73s/it, loss=0.209]

2/10 * Epoch (train):  50% 16/32 [01:38<01:31,  5.73s/it, loss=0.211]

2/10 * Epoch (train):  53% 17/32 [01:38<01:26,  5.76s/it, loss=0.211]

2/10 * Epoch (train):  53% 17/32 [01:44<01:26,  5.76s/it, loss=0.212]

2/10 * Epoch (train):  56% 18/32 [01:44<01:20,  5.77s/it, loss=0.212]

2/10 * Epoch (train):  56% 18/32 [01:50<01:20,  5.77s/it, loss=0.221]

2/10 * Epoch (train):  59% 19/32 [01:50<01:15,  5.78s/it, loss=0.221]

2/10 * Epoch (train):  59% 19/32 [01:55<01:15,  5.78s/it, loss=0.204]

2/10 *

3/10 * Epoch (train):  75% 24/32 [02:23<00:45,  5.66s/it, loss=0.216]

3/10 * Epoch (train):  78% 25/32 [02:23<00:39,  5.69s/it, loss=0.216]

3/10 * Epoch (train):  78% 25/32 [02:29<00:39,  5.69s/it, loss=0.208]

3/10 * Epoch (train):  81% 26/32 [02:29<00:34,  5.73s/it, loss=0.208]

3/10 * Epoch (train):  81% 26/32 [02:34<00:34,  5.73s/it, loss=0.214]

3/10 * Epoch (train):  84% 27/32 [02:34<00:28,  5.72s/it, loss=0.214]

3/10 * Epoch (train):  84% 27/32 [02:40<00:28,  5.72s/it, loss=0.215]

3/10 * Epoch (train):  88% 28/32 [02:40<00:22,  5.69s/it, loss=0.215]

3/10 * Epoch (train):  88% 28/32 [02:46<00:22,  5.69s/it, loss=0.196]

3/10 * Epoch (train):  91% 29/32 [02:46<00:17,  5.68s/it, loss=0.196]

3/10 * Epoch (train):  91% 29/32 [02:51<00:17,  5.68s/it, loss=0.214]

3/10 * Epoch (train):  94% 30/32 [02:51<00:11,  5.66s/it, loss=0.214]

3/10 * Epoch (train):  94% 30/32 [02:57<00:11,  5.66s/it, loss=0.211]

3/10 * Epoch (train):  97% 31/32 [02:57<00:05,  5.71s/it, loss=0.211]

3/10 *

4/10 * Epoch (test):   0% 0/2 [00:05<?, ?it/s, loss=0.218]

4/10 * Epoch (test):  50% 1/2 [00:05<00:05,  5.86s/it, loss=0.218]

4/10 * Epoch (test):  50% 1/2 [00:11<00:05,  5.86s/it, loss=0.211]

4/10 * Epoch (test): 100% 2/2 [00:11<00:00,  5.84s/it, loss=0.211]

4/10 * Epoch (test): 100% 2/2 [00:11<00:00,  5.83s/it, loss=0.211]
[2020-05-27 21:11:44,743] 
4/10 * Epoch 4 (_base): lr=0.0010 | momentum=0.9000
4/10 * Epoch 4 (train): loss=0.2106
4/10 * Epoch 4 (valid): loss=0.2096
4/10 * Epoch 4 (test): loss=0.2145
[2020-05-27 21:11:44,743] 
4/10 * Epoch 4 (_base): lr=0.0010 | momentum=0.9000
4/10 * Epoch 4 (train): loss=0.2106
4/10 * Epoch 4 (valid): loss=0.2096
4/10 * Epoch 4 (test): loss=0.2145
[2020-05-27 21:11:44,743] 
4/10 * Epoch 4 (_base): lr=0.0010 | momentum=0.9000
4/10 * Epoch 4 (train): loss=0.2106
4/10 * Epoch 4 (valid): loss=0.2096
4/10 * Epoch 4 (test): loss=0.2145


5/10 * Epoch (train):   0% 0/32 [00:00<?, ?it/s]

5/10 * Epoch (train):   0% 0/32 [00:05<?, ?it/s, loss=0.210

6/10 * Epoch (train):  16% 5/32 [00:34<02:36,  5.80s/it, loss=0.212]

6/10 * Epoch (train):  19% 6/32 [00:34<02:30,  5.78s/it, loss=0.212]

6/10 * Epoch (train):  19% 6/32 [00:40<02:30,  5.78s/it, loss=0.213]

6/10 * Epoch (train):  22% 7/32 [00:40<02:24,  5.79s/it, loss=0.213]

6/10 * Epoch (train):  22% 7/32 [00:46<02:24,  5.79s/it, loss=0.209]

6/10 * Epoch (train):  25% 8/32 [00:46<02:19,  5.80s/it, loss=0.209]

6/10 * Epoch (train):  25% 8/32 [00:52<02:19,  5.80s/it, loss=0.208]

6/10 * Epoch (train):  28% 9/32 [00:52<02:12,  5.78s/it, loss=0.208]

6/10 * Epoch (train):  28% 9/32 [00:57<02:12,  5.78s/it, loss=0.208]

6/10 * Epoch (train):  31% 10/32 [00:57<02:07,  5.79s/it, loss=0.208]

6/10 * Epoch (train):  31% 10/32 [01:03<02:07,  5.79s/it, loss=0.216]

6/10 * Epoch (train):  34% 11/32 [01:03<02:01,  5.79s/it, loss=0.216]

6/10 * Epoch (train):  34% 11/32 [01:09<02:01,  5.79s/it, loss=0.204]

6/10 * Epoch (train):  38% 12/32 [01:09<01:55,  5.79s/it, loss=0.204]

6/10 * Epoch (t

7/10 * Epoch (train):  53% 17/32 [01:38<01:26,  5.75s/it, loss=0.210]

7/10 * Epoch (train):  53% 17/32 [01:44<01:26,  5.75s/it, loss=0.211]

7/10 * Epoch (train):  56% 18/32 [01:44<01:20,  5.77s/it, loss=0.211]

7/10 * Epoch (train):  56% 18/32 [01:49<01:20,  5.77s/it, loss=0.221]

7/10 * Epoch (train):  59% 19/32 [01:49<01:15,  5.78s/it, loss=0.221]

7/10 * Epoch (train):  59% 19/32 [01:55<01:15,  5.78s/it, loss=0.204]

7/10 * Epoch (train):  62% 20/32 [01:55<01:08,  5.70s/it, loss=0.204]

7/10 * Epoch (train):  62% 20/32 [02:01<01:08,  5.70s/it, loss=0.212]

7/10 * Epoch (train):  66% 21/32 [02:01<01:03,  5.73s/it, loss=0.212]

7/10 * Epoch (train):  66% 21/32 [02:06<01:03,  5.73s/it, loss=0.201]

7/10 * Epoch (train):  69% 22/32 [02:06<00:57,  5.73s/it, loss=0.201]

7/10 * Epoch (train):  69% 22/32 [02:12<00:57,  5.73s/it, loss=0.212]

7/10 * Epoch (train):  72% 23/32 [02:12<00:51,  5.72s/it, loss=0.212]

7/10 * Epoch (train):  72% 23/32 [02:18<00:51,  5.72s/it, loss=0.204]

7/10 *

8/10 * Epoch (train):  88% 28/32 [02:46<00:22,  5.73s/it, loss=0.196]

8/10 * Epoch (train):  91% 29/32 [02:46<00:17,  5.72s/it, loss=0.196]

8/10 * Epoch (train):  91% 29/32 [02:52<00:17,  5.72s/it, loss=0.217]

8/10 * Epoch (train):  94% 30/32 [02:52<00:11,  5.70s/it, loss=0.217]

8/10 * Epoch (train):  94% 30/32 [02:58<00:11,  5.70s/it, loss=0.210]

8/10 * Epoch (train):  97% 31/32 [02:58<00:05,  5.75s/it, loss=0.210]

8/10 * Epoch (train):  97% 31/32 [03:03<00:05,  5.75s/it, loss=0.213]

8/10 * Epoch (train): 100% 32/32 [03:03<00:00,  5.70s/it, loss=0.213]

8/10 * Epoch (train): 100% 32/32 [03:03<00:00,  5.75s/it, loss=0.213]


8/10 * Epoch (valid):   0% 0/2 [00:00<?, ?it/s]

8/10 * Epoch (valid):   0% 0/2 [00:05<?, ?it/s, loss=0.213]

8/10 * Epoch (valid):  50% 1/2 [00:05<00:05,  5.96s/it, loss=0.213]

8/10 * Epoch (valid):  50% 1/2 [00:11<00:05,  5.96s/it, loss=0.206]

8/10 * Epoch (valid): 100% 2/2 [00:11<00:00,  5.94s/it, loss=0.206]

8/10 * Epoch (valid): 100% 2/2 [00:11<00:00

[2020-05-27 21:29:02,772] 
9/10 * Epoch 9 (_base): lr=0.0010 | momentum=0.9000
9/10 * Epoch 9 (train): loss=0.2100
9/10 * Epoch 9 (valid): loss=0.2097
9/10 * Epoch 9 (test): loss=0.2143


10/10 * Epoch (train):   0% 0/32 [00:00<?, ?it/s]

10/10 * Epoch (train):   0% 0/32 [00:05<?, ?it/s, loss=0.209]

10/10 * Epoch (train):   3% 1/32 [00:05<03:00,  5.81s/it, loss=0.209]

10/10 * Epoch (train):   3% 1/32 [00:11<03:00,  5.81s/it, loss=0.216]

10/10 * Epoch (train):   6% 2/32 [00:11<02:54,  5.83s/it, loss=0.216]

10/10 * Epoch (train):   6% 2/32 [00:17<02:54,  5.83s/it, loss=0.213]

10/10 * Epoch (train):   9% 3/32 [00:17<02:49,  5.83s/it, loss=0.213]

10/10 * Epoch (train):   9% 3/32 [00:23<02:49,  5.83s/it, loss=0.208]

10/10 * Epoch (train):  12% 4/32 [00:23<02:42,  5.82s/it, loss=0.208]

10/10 * Epoch (train):  12% 4/32 [00:29<02:42,  5.82s/it, loss=0.213]

10/10 * Epoch (train):  16% 5/32 [00:29<02:36,  5.81s/it, loss=0.213]

10/10 * Epoch (train):  16% 5/32 [00:34<02:36,  5.81s/it, l

In [46]:
device = torch.device("cuda")

references = []
predictions = []
model.eval()
for i, item in tqdm(enumerate(data.DataLoader(ExtDataset(ext_test_records, vocabulary, fse_model), batch_size=1, collate_fn=collate_fn)), total=ext_test_records.shape[0]):
    logits = model(item["features"].to(device))[0] # Прямой проход
    record = ext_test_records.iloc[i]
    predicted_summary = []
    for i, logit in enumerate(logits):
        if logit > 0.05:
            predicted_summary.append(record['sentences'][i])
    if not predicted_summary:
        predicted_summary.append(record['sentences'][torch.max(F.softmax(logits), dim=0)[1].item()])
#         predicted_summary.append(record['sentences'][torch.max(logits, dim=0)[1].item()])
    predicted_summary = " ".join(predicted_summary)
    references.append(record['summary'].lower())
    predictions.append(predicted_summary)

calc_scores(references, predictions)

/home/care1e55/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



/home/care1e55/.local/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.




Count: 256
Ref: атака американских военных на аэропорт багдада будет чревата тяжелыми последствиями, отметили в российском миде. прежде всего потому, что в результате этого погиб командующий силами специального назначения «аль-кудс» касем сулеймани. иранские власти предупредили сша, что «преступников ожидает суровая месть».
Hyp: убийство командующего силами специального назначения «аль-кудс» корпуса стражей исламской революции (элитные части вооруженных сил ирана) генерала касема сулеймани может привести к тяжелым последствиям для мира в регионе, считают российские дипломаты. «данный шаг вашингтона чреват тяжелыми последствиями для регионального мира и стабильности. исходим из того, что подобные акции не способствуют нахождению развязок сложных проблем, накопившихся на ближнем востоке, а напротив, ведут к новому раунду эскалации напряженности в регионе», — говорится в опубликованном в пятницу комментарии мид россии о недавней атаке сша на аэропорт в багдаде, в результате которой погиб